In [1]:
import pandas as pd

In [2]:
# !pip install pyarrow

In [3]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [4]:
df.shape #Q1 19

(3066766, 19)

Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [5]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'])
df['duration'] = df['duration'].apply(lambda td: td.total_seconds() /60)
# df = df[pd.DatetimeIndex(df['tpep_pickup_datetime']).month==1]
df.duration.std()


42.594351241920904

Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [6]:
# Dropping outliers

df_no_outliers = df[(df['duration'] >= 1) &(df['duration']<= 60)].copy()
df_no_outliers.shape[0] / df.shape[0] 

0.9812202822125979

Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)

Fit a dictionary vectorizer

Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [7]:
cat = ['PULocationID','DOLocationID']

df_no_outliers[cat] = df_no_outliers[cat].astype('str')

In [8]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
d = df_no_outliers[cat].to_dict('records')
X_train = v.fit_transform(d)

### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters

Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [9]:
from sklearn.linear_model import LinearRegression
y_train = df_no_outliers.duration.values

reg = LinearRegression()
reg.fit(X_train, y_train)


LinearRegression()

In [10]:
from sklearn.metrics import mean_squared_error

y_pred = reg.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
rmse

7.6492610279057605

### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [11]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

df_val[cat] = df_val[cat].astype('str')

df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime'])
df_val['duration'] = df_val['duration'].apply(lambda td: td.total_seconds() /60)
df_val = df_val[(df_val['duration'] >= 1) &(df_val['duration']<= 60)].copy()


In [12]:
X_val = v.transform(df_val[cat].to_dict('records'))
X_val.shape

(2855951, 515)

In [13]:
y_val = df_val.duration.values

y_pred_val = reg.predict(X_val)
rmse = mean_squared_error(y_true=y_val, y_pred=y_pred_val, squared=False)
rmse

7.81183265470218